# **Calculos Estadisticas**
Aca van a empezar los calculos de las estadisticas de lectura y NLP(PLN)

Vamos a utilizar las librerias

* [textstat](https://github.com/textstat/textstat)  : https://pypi.org/project/textstat/  
  * uso: https://www.kaggle.com/code/yhirakawa/textstat-how-to-evaluate-readability
         https://textstat.org/
         

* [textblob](https://textblob.readthedocs.io/en/dev/) : https://pypi.org/project/textblob/  
    * uso: https://pypi.org/project/textblob/  
           https://colab.research.google.com/github/vitojph/kschool-nlp-18/blob/master/notebooks/textblob.ipynb  
           https://textblob.readthedocs.io/en/dev/  
           https://notebook.community/vitojph/kschool-nlp/notebooks-py2/textblob  
           

* [vader](https://github.com/brunneis/vader-multi): https://unipython.com/analisis-de-sentimientos-con-textblob-y-vader/


https://pybonacci.org/2015/11/24/como-hacer-analisis-de-sentimiento-en-espanol-2/
https://pypi.org/project/langdetect/


**listo las funciones dir(textstat)**

[Markdown Guide](https://www.markdownguide.org "The best Markdown reference guide")

[textstat](https://github.com/textstat/textstat)




In [111]:
import warnings

# Suprimir las advertencias


import pandas as pd
import sqlite3
import textstat
import langdetect
import nltk
from nltk.corpus import stopwords
#from textblob import TextBlob


# FUNCIONES UDF


In [124]:
def contar_stopwords(texto):

    # Descargar las stopwords en español
    nltk.download('stopwords', quiet=True)

    # Obtener la lista de stopwords en español
    stopwords_es = set(stopwords.words('spanish'))
    
    # Convertir el texto a minúsculas y dividir en palabras
    palabras = texto.lower().split()
    # Contar las palabras que están en la lista de stopwords
    num_stopwords = sum(1 for palabra in palabras if palabra in stopwords_es)
    return num_stopwords

#texto = "La capital de Francia es París. ¿Sabías que París es conocida como la Ciudad de la Luz? ¡Es un lugar increíble!"
texto = "La capital de Francia es **París**."
num_stopwords = contar_stopwords(texto)
print(f"Número de stopwords en el texto: {num_stopwords}")

Número de stopwords en el texto: 3


In [105]:
#from ftlangdetect import detect
##result = detect(text="Martian Chronicles by Ray Bradbury is a collection of stories exploring human colonization on Mars. With poetic prose and a visionary imagination, Bradbury weaves emotive tales of solitude, nostalgia, and the human nature through imagined encounters between colonists and Martians. Each story reveals the fragility of existence and the search for meaning in an alien world, while reflecting parallels with the human condition on Earth. This science fiction masterpiece captivates with its evocative style and its ability to transcend technology, delving into the deepest depths of the human soul.", low_memory=False)
#detect(text="La capital de Francia es París (en francés, Paris).", low_memory=False)['lang']
##print(result['lang'])
##result = detect(text="Bugün hava çok güzel", low_memory=True)
##print(result)


'es'

# **MAIN**

In [106]:
# conexion a la BBDD
path = "/mnt/e/trabajo/sysberisso/GIT_REPOS/LLM_Benchmark/prompts/db"
con = sqlite3.connect(f'{path}/prompt_sqlite.db')


In [127]:
# consulto los objetos de la BBDD para obtener los nombres de los modelos gemma_7b y llama2
df_view = pd.read_sql_query("SELECT name FROM sqlite_master where type='view' and name not like 'extra_%' limit 2" , con)
df_view

,name
0,gemma_7b_data1
1,llama2_data1


In [128]:
#for index, row in df_view.iterrows():
#  print(f"\n#### {row['name']} ####\n")
name_tbl = df_view.iloc[1]['name']
print(name_tbl)
sql_query = f"SELECT id_promtp, answers FROM {name_tbl}" #f"SELECT id_promtp, answers FROM {row['name']}"
df_prompts = pd.read_sql_query(sql_query, con)

df_prompts

llama2_data1


,id_promtp,answers
0,1,\n¡Buenos días! ¡Claro que sí! Estoy aquí para...
1,2,\nLa capital de Francia es París.
2,3,\n¡Por supuesto! Argentina es un país fascinan...
3,4,\nInteligencia Artificial (IA) es el campo de ...
4,5,\nSure! Here is the translation of the text in...
5,6,"\nEl aire es libre, la brisa del mar\nUna sens..."
6,7,"Título: ""The Audition""\n\nAct 1:\n\nThe scene ..."
7,8,"```\nprint(""Hola, mundo!"")\n```\nEste código u..."
8,9,\n ¡Claro! Para analizar este conjunto de dato...
9,10,\nPuedes crear una función en PySpark utilizan...


# GEMINI
La biblioteca textstat ofrece una variedad de funciones para analizar textos en español. Estas funciones se pueden utilizar para evaluar la legibilidad, la complejidad y otras características lingüísticas del texto. A continuación, se presenta un resumen de algunas de las funciones más relevantes para el análisis de textos en español:

Funciones de legibilidad:

    flesch_reading_ease(): Calcula la facilidad de lectura del texto según la fórmula de Flesch. Los valores más altos indican un texto más fácil de leer.
    flesch_kincaid_grade_level(): Calcula el nivel de grado del texto según la fórmula de Flesch-Kincaid. Los valores más altos indican un texto más difícil de leer.
    gunning_fog_index(): Calcula la complejidad del texto según la fórmula de Gunning Fog. Los valores más altos indican un texto más complejo.
    coleman_liau_index(): Calcula la legibilidad del texto según la fórmula de Coleman-Liau. Los valores más altos indican un texto más difícil de leer.
    automated_readability_index(): Calcula la legibilidad del texto según la fórmula de Automated Readability Index. Los valores más altos indican un texto más difícil de leer.

Funciones de complejidad:

    lexical_density(): Calcula la densidad léxica del texto, es decir, la proporción de palabras únicas por cada 100 palabras. Los valores más altos indican un texto más complejo.
    avg_sentence_length(): Calcula la longitud promedio de las oraciones en el texto. Los valores más altos indican oraciones más largas y complejas.
    avg_word_length(): Calcula la longitud promedio de las palabras en el texto. Los valores más altos indican palabras más largas y complejas.
    type_token_ratio(): Calcula la proporción de palabras únicas por cada 100 palabras totales. Los valores más altos indican un mayor uso de vocabulario.

Otras funciones:

    count_words(): Cuenta el número de palabras en el texto.
    count_sentences(): Cuenta el número de oraciones en el texto.
    count_syllables(): Cuenta el número de sílabas en el texto.
    count_characters(): Cuenta el número de caracteres en el texto.


# CHATGPT
    Conteo de caracteres y palabras:
        textstat.char_count(text): Cuenta el número de caracteres en el texto.
        textstat.lexicon_count(text, removepunct=True): Cuenta el número de palabras en el texto.

    Conteo de oraciones:
        textstat.sentence_count(text): Cuenta el número de oraciones en el texto.

    Longitud promedio de palabras y oraciones:
        textstat.avg_sentence_length(text): Calcula la longitud promedio de las oraciones en el texto.
        textstat.avg_letter_per_word(text): Calcula el número promedio de letras por palabra.

    Diversas métricas de legibilidad:
        textstat.flesch_reading_ease(text): Calcula la facilidad de lectura de Flesch (aunque la fórmula original está optimizada para inglés).
        textstat.smog_index(text): Calcula el índice SMOG (Simplified Measure of Gobbledygook).
        textstat.coleman_liau_index(text): Calcula el índice de Coleman-Liau.
        textstat.automated_readability_index(text): Calcula el índice de legibilidad automatizado.

    Índices y niveles de grado:
        textstat.linsear_write_formula(text): Calcula la fórmula de Linsear Write.
        textstat.gunning_fog(text): Calcula el índice Gunning Fog.
        textstat.dale_chall_readability_score(text): Calcula la puntuación de legibilidad de Dale-Chall (requiere una lista de palabras comunes en inglés, por lo que puede no ser precisa para español).

In [130]:
#%%capture

# detecto el lenguaje del texto
df_prompts['lang']=df_prompts['answers'].apply(lambda x: langdetect.detect(x)) 

#  devuelve el tiempo de lectura de un texto
df_prompts['reading_time']=df_prompts['answers'].apply(lambda x: textstat.reading_time(x))

#######################################################
##Índice de lecturabilidad Fernandez-Huerta (Spanish) : https://legible.es/blog/lecturabilidad-fernandez-huerta/
#df_prompts['f-h_index']=df_prompts['answers'].apply(lambda x: textstat.fernandez_huerta(x))
#
##Índice de perspicuidad de Szigriszt-Pazos (Spanish) : https://legible.es/blog/perspicuidad-szigriszt-pazos/
#df_prompts['s-p_index']=df_prompts['answers'].apply(lambda x: textstat.szigriszt_pazos(x))
#
##Fórmula de comprensibilidad de Gutiérrez de Polini (Spanish) : https://legible.es/blog/comprensibilidad-gutierrez-de-polini/
#df_prompts['g-po_index']=df_prompts['answers'].apply(lambda x: textstat.gutierrez_polini(x))
#
##Fórmula de Crawford (Spanish) : https://legible.es/blog/formula-de-crawford/
#df_prompts['crowford_index']=df_prompts['answers'].apply(lambda x: textstat.crawford(x))
#######################################

# Aggregates and Averages


#Sentence Count: Returns the number of sentences present in the given text.
df_prompts['sentence_count']=df_prompts['answers'].apply(lambda x: textstat.sentence_count(x))
#Character Count: Returns the number of characters present in the given text. la puntuacion incluida
df_prompts['char_count']=df_prompts['answers'].apply(lambda x: textstat.char_count(x))
#Letter Count: Returns the number of characters present in the given text without punctuation.
df_prompts['letter_count']=df_prompts['answers'].apply(lambda x: textstat.letter_count(x))
#Syllable Count: Returns the number of syllables present in the given text
df_prompts['lexicon_count']=df_prompts['answers'].apply(lambda x: textstat.lexicon_count(x))
#avg_sentence_length: Calcula la longitud promedio de las oraciones en el texto.
df_prompts['avg_sentence_length']=df_prompts['answers'].apply(lambda x: textstat.avg_sentence_length(x))
#avg_letter_per_word: Calcula el número promedio de letras por palabra
df_prompts['avg_letter_per_word']=df_prompts['answers'].apply(lambda x: textstat.avg_letter_per_word(x))

# stop words function
df_prompts['qtty_stopwords']=df_prompts['answers'].apply(contar_stopwords)
df_prompts




,id_promtp,answers,lang,reading_time,sentence_count,char_count,letter_count,lexicon_count,avg_sentence_length,avg_letter_per_word,contar_stopwords,qtty_stopwords
0,1,\n¡Buenos días! ¡Claro que sí! Estoy aquí para...,es,2.60,3,177,169,39,13.0,4.33,21,21
1,2,\nLa capital de Francia es París.,es,0.38,1,26,25,6,6.0,4.17,3,3
2,3,\n¡Por supuesto! Argentina es un país fascinan...,es,29.10,25,1981,1918,406,16.2,4.72,168,168
3,4,\nInteligencia Artificial (IA) es el campo de ...,es,18.36,10,1250,1224,227,22.7,5.39,99,99
4,5,\nSure! Here is the translation of the text in...,en,8.04,4,547,532,96,24.0,5.54,1,1
5,6,"\nEl aire es libre, la brisa del mar\nUna sens...",es,11.43,1,778,764,180,180.0,4.24,99,99
6,7,"Título: ""The Audition""\n\nAct 1:\n\nThe scene ...",en,24.91,22,1696,1628,361,16.4,4.51,21,21
7,8,"```\nprint(""Hola, mundo!"")\n```\nEste código u...",es,10.30,8,701,645,125,15.6,5.16,53,53
8,9,\n ¡Claro! Para analizar este conjunto de dato...,es,20.18,13,1374,1232,246,18.9,5.01,102,102
9,10,\nPuedes crear una función en PySpark utilizan...,es,18.32,14,1247,1117,202,14.4,5.53,82,82
